# Credit Modelling

**Modeling a borrower's [credit risk](https://en.wikipedia.org/wiki/Credit_risk)**

### Problem Statement
   - **Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?**

I'll be working with financial lending data from [Lending Club](https://www.lendingclub.com/). 
Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower.

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiving a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the [origination fee](https://help.lendingclub.com/hc/en-us/articles/214501207-What-is-the-origination-fee-) that Lending Club charges.


The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors.

![](how-social-lending-works.jpg)

However, and some borrowers **default** on the loan

**In this project, I'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time.**

### Data Dictonary and getting familiar with features
Before we start with our Model, We'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not

Lending Club releases data for all of the approved and declined loan applications periodically on their [website](https://www.lendingclub.com/info/download-data.action)

The **LoanStats** sheet describes the approved loans datasets and the **RejectStats** describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only

Here's a link to the data dictionary file hosted on [Google Drive](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit)

### Importing and Reading the data

We'll focus on approved loans data from 2007 to 2011, since a good number of the loans have already finished. And as for later years, many of the loans are current and still being paid off.

In [2]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning) #Ignore Warning messages

import pandas as pd 

loans_2007 = pd.read_csv('loans_2007.csv')
loans_2007.drop_duplicates()
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

C:\Users\AMAN3001\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

For now lets pay attention to any features that:

 - leak information from the future (after the loan has already been funded)
 - don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
 - formatted poorly and need to be cleaned up
 - require more data or a lot of processing to turn into a useful feature
 - contain redundant information
 
 I created a table that contains the name, data type, first row's value, and description from the data dictionary for the first 18 columns.
 
 
 | name                | dtype   | first value | description                                                                                                                       |
|---------------------|---------|-------------|-----------------------------------------------------------------------------------------------------------------------------------|
| id                  | object  | 1077501     | A unique LC assigned ID for the loan listing.                                                                                     |
| member_id           | float64 | 1.2966e+06  | A unique LC assigned Id for the borrower member.                                                                                  |
| loan_amnt           | float64 | 5000        | The listed amount of the loan applied for by the borrower.                                                                        |
| funded_amnt         | float64 | 5000        | The total amount committed to that loan at that point in time.                                                                    |
| funded_amnt_inv     | float64 | 49750       | The total amount committed by investors for that loan at that point in time.                                                      |
| term                | object  | 36 months   | The number of payments on the loan. Values are in months and can be either 36 or 60.                                              |
| int_rate            | object  | 10.65%      | Interest Rate on the loan                                                                                                         |
| installment         | float64 | 162.87      | The monthly payment owed by the borrower if the loan originates.                                                                  |
| grade               | object  | B           | LC assigned loan grade                                                                                                            |
| sub_grade           | object  | B2          | LC assigned loan subgrade                                                                                                         |
| emp_title           | object  | NaN         | The job title supplied by the Borrower when applying for the loan.                                                                |
| emp_length          | object  | 10+ years   | Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. |
| home_ownership      | object  | RENT        | The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.               |
| annual_inc          | float64 | 24000       | The self-reported annual income provided by the borrower during registration.                                                     |
| verification_status | object  | Verified    | Indicates if income was verified by LC, not verified, or if the income source was verified                                        |
| issue_d             | object  | Dec-2011    | The month which the loan was funded                                                                                               |
| loan_status         | object  | Charged Off | Current status of the loan                                                                                                        |
| pymnt_plan          | object  | n           | Indicates if a payment plan has been put in place for the loan                                                                    |
| purpose             | object  | car         | A category provided by the borrower for the loan request.                                                                         |

After analyzing each column, I conclude that the following features need to be removed based on the 5 criteria above:

 - id: randomly generated field by Lending Club for unique identification purposes only
 - member_id: also a randomly generated field by Lending Club for unique identification purposes only
 - funded_amnt: leaks data from the future (after the loan is already started to be funded)
 - funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
 - grade: contains redundant information as the interest rate column (int_rate)
 - sub_grade: also contains redundant information as the interest rate column (int_rate)
 - emp_title: requires other data and a lot of processing to potentially be useful
 - issue_d: leaks data from the future (after the loan is already completed funded)

### Droping unwanted columns 1

In [3]:
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

**Lets look at next 18 columns**

| name                | dtype   | first value | description                                                                                                                                                                                              |
|---------------------|---------|-------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| title               | object  | Computer    | The loan title provided by the borrower                                                                                                                                                                  |
| zip_code            | object  | 860xx       | The first 3 numbers of the zip code provided by the borrower in the loan application.                                                                                                                    |
| addr_state          | object  | AZ          | The state provided by the borrower in the loan application                                                                                                                                               |
| dti                 | float64 | 27.65       | A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income. |
| delinq_2yrs         | float64 | 0           | The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years                                                                                             |
| earliest_cr_line    | object  | Jan-1985    | The month the borrower's earliest reported credit line was opened                                                                                                                                        |
| inq_last_6mths      | float64 | 1           | The number of inquiries in past 6 months (excluding auto and mortgage inquiries)                                                                                                                         |
| open_acc            | float64 | 3           | The number of open credit lines in the borrower's credit file.                                                                                                                                           |
| pub_rec             | float64 | 0           | Number of derogatory public records                                                                                                                                                                      |
| revol_bal           | float64 | 13648       | Total credit revolving balance                                                                                                                                                                           |
| revol_util          | object  | 83.7%       | Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.                                                                               |
| total_acc           | float64 | 9           | The total number of credit lines currently in the borrower's credit file                                                                                                                                 |
| initial_list_status | object  | f           | The initial listing status of the loan. Possible values are – W, F                                                                                                                                       |
| out_prncp           | float64 | 0           | Remaining outstanding principal for total amount funded                                                                                                                                                  |
| out_prncp_inv       | float64 | 0           | Remaining outstanding principal for portion of total amount funded by investors                                                                                                                          |
| total_pymnt         | float64 | 5863.16     | Payments received to date for total amount funded                                                                                                                                                        |
| total_pymnt_inv     | float64 | 5833.84     | Payments received to date for portion of total amount funded by investors                                                                                                                                |
| total_rec_prncp     | float64 | 5000        | Principal received to date                                                                                                                                                                               |

Within this group of columns, we need to drop the following columns:

 - zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
 - out_prncp: leaks data from the future, (after the loan already started to be paid off)
 - out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
 - total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
 - total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
 - total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)
 
**This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.**

### Droping Unwanted columns -2

In [4]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

**Lets look at last group of columns**

| name                       | dtype   | first value | description                                                                                          |
|----------------------------|---------|-------------|------------------------------------------------------------------------------------------------------|
| total_rec_int              | float64 | 863.16      | Interest received to date                                                                            |
| total_rec_late_fee         | float64 | 0           | Late fees received to date                                                                           |
| recoveries                 | float64 | 0           | post charge off gross recovery                                                                       |
| collection_recovery_fee    | float64 | 0           | post charge off collection fee                                                                       |
| last_pymnt_d               | object  | Jan-2015    | Last month payment was received                                                                      |
| last_pymnt_amnt            | float64 | 171.62      | Last total payment amount received                                                                   |
| last_credit_pull_d         | object  | Jun-2016    | The most recent month LC pulled credit for this loan                                                 |
| collections_12_mths_ex_med | float64 | 0           | Number of collections in 12 months excluding medical collections                                     |
| policy_code                | float64 | 1           | publicly available policy_code=1 new products not publicly available policy_code=2                   |
| application_type           | object  | INDIVIDUAL  | Indicates whether the loan is an individual application or a joint application with two co-borrowers |
| acc_now_delinq             | float64 | 0           | The number of accounts on which the borrower is now delinquent.                                      |
| chargeoff_within_12_mths   | float64 | 0           | Number of charge-offs within 12 months                                                               |
| delinq_amnt                | float64 | 0           | The past-due amount owed for the accounts on which the borrower is now delinquent.                   |
| pub_rec_bankruptcies       | float64 | 0           | Number of public record bankruptcies                                                                 |
| tax_liens                  | float64 | 0           | Number of tax liens                                                                                  |

In the last group of columns, we need to drop the following columns:

 - total_rec_int: leaks data from the future, (after the loan already started to be paid off),
 - total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
 - recoveries: also leaks data from the future, (after the loan already started to be paid off),
 - collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
 - last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
 - last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).
 
All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

### Droping unwanted columns - 3

In [5]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

### Deciding Target Column for our Model

We should use the **loan_status** column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower

In [6]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


**Explanation for each value in loan_status column as well as their counts**

| Loan Status                                         | Count | Meaning                                                                                                                                           |
|-----------------------------------------------------|-------|---------------------------------------------------------------------------------------------------------------------------------------------------|
| Fully Paid                                          | 33136 | Loan has been fully paid off.                                                                                                                     |
| Charged Off                                         | 5634  | Loan for which there is no longer a reasonable expectation of further payments.                                                                   |
| Does not meet the credit policy. Status:Fully Paid  | 1988  | While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.    |
| Does not meet the credit policy. Status:Charged Off | 761   | While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace. |
| In Grace Period                                     | 20    | The loan is past due but still in the grace period of 15 days.                                                                                    |
| Late (16-30 days)                                   | 8     | Loan hasn't been paid in 16 to 30 days (late on the current payment).                                                                             |
| Late (31-120 days)                                  | 24    | Loan hasn't been paid in 31 to 120 days (late on the current payment).                                                                            |
| Current                                             | 961   | Loan is up to date on current payments.                                                                                                           |
| Default                                             | 3     | Loan is defaulted on and no payment has been made for more than 121 days.                                                                         |

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the **Fully Paid** and **Charged Off** values describe the final outcome of the loan.

Now we can treat the problem as a **binary classification** one. Let's remove all the loans that don't contain either **Fully Paid** and **Charged Off** as the loan's status and then transform the Fully Paid values to **1** for the positive case and the Charged Off values to **0** for the negative case

In [7]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]

status_replace = {
    'loan_status': {
        'Fully Paid': 1,
        'Charged Off': 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

## Dealing with Class Imbalance
While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off.

 - This class imbalance will lead our model to be bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations
### Removing columns with only one unique value
These columns won't be useful for the model since they don't add any information to each loan application

In [8]:
orig_columns = loans_2007.columns
drop_columns = []

#We drop all missing values and find unique values for each columns

for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    
    #If unique value in column is 1 we Drop that column
    if len(col_series) == 1:
        drop_columns.append(col)

        
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


we were able to remove 9 more columns since they only contained 1 unique value

### Dealing with Missing Values

In [9]:
# Finding Columns with null values 
null_counts = loans_2007.isnull().sum()
print(null_counts[null_counts>0])

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


Let's inspect the values of the column **pub_rec_bankruptcies**

In [10]:
print(loans_2007.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


This column offers very little variability, nearly 94% of values are in the same category. It probably won't have much predictive value. We'll **Drop** the column

For rest of the Columns:
 - **Title, revol_util, last_credit_pull_d**: As they have less than 50 missing values we will drop the missing value rows
 - **emp_length**: Domain knowledge tells us that employment length is frequently used in assessing how risky a potential borrower is, so we'll keep this column despite its relatively large amount of missing values.

In [11]:
#Drop column pub_rec_bankruptcies

loans_2007 = loans_2007.drop('pub_rec_bankruptcies', axis=1)
loans_2007 = loans_2007.dropna(axis=0)
print(loans_2007.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


### Converting Object columns to numberics
To run ML models the object columns that contain text need to be converted to numerical data types.

In [12]:
#Select only the object columns into new df
object_columns_df = loans_2007.select_dtypes(include=['object'])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


**Date Value Columns**
 - **earliest_cr_line**: The month the borrower's earliest reported credit line was opened
 - **last_credit_pull_d**: The most recent month Lending Club pulled credit for this loan

**Numberic Value Representing Columns**
 - **int_rate**: interest rate of the loan in %
 - **revol_util**: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit, read more [here](http://blog.credit.com/2013/04/what-is-revolving-utilization-65530/)
 
**Categorical Values Columns**
 - **home_ownership**: home ownership status, can only be 1 of 4 categorical values according to the data dictionary
 - **verification_status**: indicates if income was verified by Lending Club
 - **emp_length**: number of years the borrower was employed upon time of application
 - **term**: number of payments on the loan, either 36 or 60
 - **addr_state**: borrower's state of residence
 - **purpose**: a category provided by the borrower for the loan request
 - **title**: loan title provided by the borrower
 
 Let's explore the unique value counts of the columns that seem like they contain categorical values

In [17]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for c in cols:
    print(loans_2007[c].value_counts())
    print('-'*40)

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
----------------------------------------
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
----------------------------------------
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
----------------------------------------
 36 months    28234
 60 months     9441
Name: term, dtype: int64
----------------------------------------
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK   

We should clean the **emp_length** column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years)

Let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.

In [19]:
print(loans_2007["title"].value_counts())
print('-'*40)
print(loans_2007["purpose"].value_counts())

Debt Consolidation                     2068
Debt Consolidation Loan                1599
Personal Loan                           624
Consolidation                           488
debt consolidation                      466
Credit Card Consolidation               345
Home Improvement                        336
Debt consolidation                      314
Small Business Loan                     298
Credit Card Loan                        294
Personal                                290
Consolidation Loan                      250
Home Improvement Loan                   228
personal loan                           219
Loan                                    202
Wedding Loan                            199
personal                                198
Car Loan                                188
consolidation                           186
Other Loan                              168
Wedding                                 148
Credit Card Payoff                      144
Credit Card Refinance           

It seems like the **purpose** and **title** columns do contain overlapping information but we'll **keep** the **purpose** column since it contains a few discrete values.


We can use the following mapping to clean the **emp_length** column:

 - "10+ years": 10
 - "9 years": 9
 - "8 years": 8
 - "7 years": 7
 - "6 years": 6
 - "5 years": 5
 - "4 years": 4
 - "3 years": 3
 - "2 years": 2
 - "1 year": 1
 - "< 1 year": 0
 - "n/a": 0
 
 With Assumptions that:
  - people who may have been working more than 10 years have only really worked for 10 years
  - people who've worked less than a year or if the information is not available that they've worked for 0
  
We'll also remove the **last_credit_pull_d, addr_state, title, and earliest_cr_line** columns from loans and

Convert the **int_rate and revol_util** columns to float columns

In [22]:
#Remove unwanted columns
loans = loans_2007.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)

#Convert rate and revol_util to float
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')
#Clean emp_length column

mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.replace(mapping_dict)
loans.head(2)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0


### Dummy Variables
Let's now encode the **home_ownership, verification_status, purpose, and term** columns as dummy variables so we can use them in our model

In [23]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]

#Return Dataframe containing 1 column for each dummy variable
dummy_df = pd.get_dummies(loans[cat_columns])

#use the concat method to add these dummy columns back to the original Dataframe
loans = pd.concat([loans, dummy_df], axis=1)

#drop the original columns entirely using the drop method
loans = loans.drop(cat_columns, axis=1)

loans.head(2)


,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,1


We have completed the necessary data preprations required to train the ML models.

In [24]:
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              37675 non-null float64
int_rate                               37675 non-null float64
installment                            37675 non-null float64
emp_length                             37675 non-null object
annual_inc                             37675 non-null float64
loan_status                            37675 non-null int64
dti                                    37675 non-null float64
delinq_2yrs                            37675 non-null float64
inq_last_6mths                         37675 non-null float64
open_acc                               37675 non-null float64
pub_rec                                37675 non-null float64
revol_bal                              37675 non-null float64
revol_util                             37675 non-null float64
total_acc                              37675 non-null float64
home_ownership_MORTGAGE   

# Defining our Error Metric

**Our objective**- Fund enough loans that are paid off on time to offset our losses from loans that aren't paid off.

We're primarily concerned with false positives and false negatives:
 - **False Positives**- We predict that a loan will be paid off on time, but it actually isn't
 - **False Negative** - we predict that a loan won't be paid off on time, but it actually would be paid off on time
 
we should optimize for:

 - high [recall](https://en.wikipedia.org/wiki/Precision_and_recall#Recall) (true positive rate)
   
 - low [fall-out](https://en.wikipedia.org/wiki/Information_retrieval#Fall-out) (false positive rate)

# Applying Logistic Regression

We'll use K-Fold Cross validation to check out model accuracy

Also to deal with Class Impbalance we will penalize misclassifications of the less prevalent class(loan_status=0) more than the other class (loan_status=1)

We can do this by setting the **class_weight** parameter to penalty when creating the LogisticRegression instance

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

#Set Class weight to 10 to deal with Class Imbalance and penalize False Positives more
penalty = {
    0:10,
    1:1
}
lr = LogisticRegression(class_weight=penalty)
cols = loans.columns
train_cols = cols.drop('loan_status')
features = loans[train_cols]
target = loans['loan_status']

# Make predictions using 3-fold cross-validation.
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print('True Positive Rate:', tpr)
print('False Positive Rate:', fpr)


True Positive Rate: 0.2305977975878343
False Positive Rate: 0.22724580454096743


It looks like assigning manual penalties lowered the false positive rate to 22%, and thus lowered our risk.

Note that this comes at the expense of true positive rate.

# Applying Random Forest

In [42]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight="balanced", random_state=1)

# Make predictions using 3-fold cross-validation.
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print('True Positive Rate:', tpr)
print('False Positive Rate:', fpr)

True Positive Rate: 0.9630637126376508
False Positive Rate: 0.9636722606120435


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s